# Ensemble Evaluation (Offline)

This notebook evaluates the local stub ML vs the ensemble.

Instructions:
- Place local videos under `examples/` and optional `examples/stub_rules.json`.
- No downloads are performed. Provide your own dataset paths below.



In [ ]:
from pathlib import Path
import json
import numpy as np
from deepforensics.app import config, ingest, prnu as prnu_mod, metadata as metadata_mod, ml as ml_mod, ensemble as ensemble_mod

# Configure dataset
DATA_DIR = Path('examples')
VIDEOS = [p for p in DATA_DIR.glob('*.mp4')]
print('Found videos:', [p.name for p in VIDEOS])



In [ ]:
results = []
for vid in VIDEOS:
    frames, _ = ingest.sample_frames(vid, Path('work')/('nb_frames_'+vid.stem))
    clip_prnu, residuals = prnu_mod.process_frames_for_prnu(frames)
    face_scores, _ = prnu_mod.face_region_scores_and_heatmaps(frames, residuals, Path('work')/'nb_evidence')
    face_scores_json = [{"frame_index": s.frame_index, "bbox": list(s.bbox), "score": s.score} for s in face_scores]
    details, flags, meta_score = metadata_mod.analyze(vid)
    ml_out = ml_mod.stub_predict(vid, flags, face_scores_json)
    prnu_similarity = 1.0 - max([s['score'] for s in face_scores_json], default=0.0)
    ens = ensemble_mod.score_and_decide(ml_out['score'], prnu_similarity, meta_score)
    results.append({"name": vid.name, "ml": ml_out['score'], "ens": ens['weighted_score']})
results
